# TensorFlow实现双层CNN
使用了两个卷积层、两个池化层、一个全连接层

In [11]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# 读入mnist数据,以独热向量读入
mnist = input_data.read_data_sets("./data/MNIST_data/", one_hot=True)
img = mnist.train.images[500]
print(img.shape)
x = tf.placeholder(tf.float32,[None, 784])
y_ = tf.placeholder(tf.float32,[None, 10])

Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz
(784,)


In [12]:
#784-->28*28
x_image = tf.reshape(x,[-1,28,28,1])

In [13]:
def weight_variable(shape):
    '''
    初始化权重w
    '''
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    '''
    初始化偏置b
    '''
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
    
def conv2d(x, W):
    '''
    实现卷积层
    '''
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    '''
    实现池化层
    '''
    #padding='SAME'表示通过填充0，使得输入和输出的形状一致
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [14]:
# 第一层 -> 卷积层
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)#池化

# 第二层 -> 卷积层
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)#池化

In [15]:
# 全连接层
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fcl_drop = tf.nn.dropout(h_fc1, keep_prob)

In [16]:
#输出层
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fcl_drop, W_fc2)+b_fc2

In [17]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
#也可以将0.0001写成le-4
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

In [18]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [19]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        # 每100轮计算一次准确率
        if i%100 == 0:
            #训练时的精度
            train_accuracy = sess.run(accuracy, feed_dict={x:batch[0],y_:batch[1], keep_prob:1.0})
            print('step %d, training accuracy %g'%(i, train_accuracy))
        sess.run(train_step,feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})
    test_accuracy = sess.run(accuracy, feed_dict={x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0})
print('测试集准确率: %g'%test_accuracy)

step 0, training accuracy 0.12
step 100, training accuracy 0.86
step 200, training accuracy 0.88
step 300, training accuracy 0.92
step 400, training accuracy 0.92
step 500, training accuracy 0.96
step 600, training accuracy 0.98
step 700, training accuracy 0.98
step 800, training accuracy 0.98
step 900, training accuracy 1
step 1000, training accuracy 0.92
step 1100, training accuracy 0.94
step 1200, training accuracy 0.98
step 1300, training accuracy 0.96
step 1400, training accuracy 1
step 1500, training accuracy 0.96
step 1600, training accuracy 1
step 1700, training accuracy 1
step 1800, training accuracy 1
step 1900, training accuracy 0.94
step 2000, training accuracy 0.98
step 2100, training accuracy 1
step 2200, training accuracy 0.96
step 2300, training accuracy 1
step 2400, training accuracy 1
step 2500, training accuracy 1
step 2600, training accuracy 1
step 2700, training accuracy 1
step 2800, training accuracy 0.96
step 2900, training accuracy 1
step 3000, training accuracy